In [3]:
import torch
import torch.nn as nn
import pandas as pd
# Prediction script (load the model)
from model import biLSTM
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [4]:
# Load the trained model
model = torch.load("/home/esbenlykke/projects/sleep_study/models/lstm_model.pt")

device = 'cpu'

model.to(device)
model.eval()



biLSTM(
  (lstm): LSTM(68, 128, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=5, bias=True)
)

In [5]:
# load test tensors
test_predictors = torch.load("/home/esbenlykke/projects/sleep_study/data/data_for_modelling/lstm/test_predictors.pt")
test_labels = torch.load("/home/esbenlykke/projects/sleep_study/data/data_for_modelling/lstm/test_labels.pt")

test_predictors = test_predictors.to(device)
test_labels = test_labels.to(device).long() + 1

# Create a TensorDataset from your tensors
test_data = TensorDataset(test_predictors, test_labels)

# Create data loader
batch_size = 64

test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [7]:
sequence_length = 20
input_size = 68

test_predictors = test_predictors.view(-1, sequence_length, input_size)

print(test_predictors.size())

torch.Size([122749, 20, 68])


In [8]:
# Make predictions
predictions = []

# Set the model to evaluation mode
model.eval()

# Wrap the test_loader with tqdm for progress tracking
with tqdm(test_loader, desc='Predicting', unit='batch') as progress_bar:
    for batch in progress_bar:
        inputs, _ = batch
        inputs = inputs.to(device)

        # Forward pass
        outputs = model(inputs)

        # Process the outputs as needed
        # ...

        # Append the batch predictions to the list
        predictions.append(outputs.detach().cpu())

        # Free GPU memory
        del inputs, outputs
        torch.cuda.empty_cache()

# Concatenate the predictions from all batches
predictions = torch.cat(predictions, dim=0)



Predicting: 100%|██████████| 1918/1918 [00:30<00:00, 62.19batch/s]


In [9]:
print(pd.DataFrame(predictions))

               0         1         2         3         4
0       0.973478  4.396975 -2.361055 -0.293792 -9.054585
1      -2.176204  8.162628 -3.329409 -4.016507 -4.837631
2       0.006118 -0.700682  4.636651 -1.138596 -4.667510
3       1.064802  3.851015 -1.907723 -0.145548 -9.054010
4      -3.584435  9.581444 -3.169254 -4.420667 -5.615054
...          ...       ...       ...       ...       ...
122744 -0.551900  0.632312 -2.195987  4.043622 -5.063848
122745 -3.049251  9.292664 -3.364517 -4.270768 -5.685663
122746 -3.516284  9.285416 -2.604633 -4.389318 -5.462536
122747 -1.188794  6.851342 -2.640457 -3.782547 -4.427862
122748  2.256259  0.198474 -3.780382  3.740668 -5.744560

[122749 rows x 5 columns]


In [10]:
# Convert the predicted outputs to the desired format (e.g., classes, probabilities, etc.)
# Perform any necessary post-processing steps on the outputs

# Example: Convert the outputs to class labels
predicted_classes = torch.argmax(predictions, dim=1)

In [17]:
print(pd.DataFrame(predicted_classes).describe)

<bound method NDFrame.describe of         0
0       1
1       1
2       2
3       1
4       1
...    ..
122744  3
122745  1
122746  1
122747  1
122748  3

[122749 rows x 1 columns]>


In [28]:
# Example: Convert the outputs to probabilities
softmax = nn.Softmax(dim=1)
probabilities = softmax(outputs)

# Create a new DataFrame with the predicted results
# Assume 'predictions_df' is a new DataFrame to store the predictions
predictions_df = pd.DataFrame({
    "Predicted_Class": predicted_classes.tolist(),
    "Probability_Class_0": probabilities[:, 0].tolist(),
    "Probability_Class_1": probabilities[:, 1].tolist(),
    # Add more columns as needed
})

# Use the predictions for further analysis or save them to a file
predictions_df.to_csv("predictions.csv", index=False)

torch.Size([41195, 20, 68])
torch.Size([41195])
torch.Size([122749, 20, 68])
torch.Size([122749])


In [20]:
print(test_predictors.shape)
out, _ = model.lstm(test_predictors)


torch.Size([122749, 20, 68])


NotImplementedError: Could not run 'aten::mkldnn_rnn_layer' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::mkldnn_rnn_layer' is only available for these backends: [CPU, Meta, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMeta, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradNestedTensor, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, Pytho